# 2 start base pods on Azure AKS

change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/core"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/core
```

## load environment variables

In [ ]:
source ${PJ_ROOT}/docs/environments/azure_aks/env

## setup alias

In [ ]:
if [ "$(uname)" == 'Darwin' ]; then
  alias randomstr32='cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 32'
elif [ "$(expr substr $(uname -s) 1 5)" == 'Linux' ]; then
  alias randomstr32='cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 32'
else
  echo "Your platform ($(uname -a)) is not supported."
  exit 1
fi

## create wildcard TLS cert files using Let's Encrypt

### create tls cert files using let's encrypt

_Outside of this notebook_
1. open a ternminal.
1. run a command displayed below.

In [ ]:
echo "docker run -it -v ${PJ_ROOT}/secrets:/etc/letsencrypt certbot/certbot certonly --manual --domain *.${DOMAIN} --email ${EMAIL} --no-eff-email --agree-tos --manual-public-ip-logging-ok --preferred-challenges dns-01 --server https://acme-v02.api.letsencrypt.org/directory"

_Outside of this notebook_
1. wait until to add a DNS TXT Record like below.

notice) **replace XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX with the 'DNS TXT record' value displayed in opened terminal.**

In [ ]:
export DNS_TXT="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [ ]:
az network dns record-set txt add-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "_acme-challenge" --value "${DNS_TXT}"

_Outside of this notebook_
1. in order to continue creating cert file process, press enter key in the opened terminal.
1. confirm to be displayed the below message at the opened terminal.
```
Congratulations! Your certificate and chain have been saved at:
```

In [ ]:
az network dns record-set txt remove-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "_acme-challenge" --value "${DNS_TXT}"

## start RabbitMQ cluster on AKS
[rabbitmq](https://www.rabbitmq.com/)

### register cert files to kubernetes secrets

In [ ]:
kubectl create secret generic rabbitmq-certifications --from-file=${PJ_ROOT}/secrets/live/${DOMAIN}/fullchain.pem --from-file=${PJ_ROOT}/secrets/live/${DOMAIN}/cert.pem --from-file=${PJ_ROOT}/secrets/live/${DOMAIN}/privkey.pem

### start RabbitMQ

In [ ]:
kubectl apply -f rabbitmq/rabbitmq-rbac.yaml

In [ ]:
kubectl apply -f rabbitmq/rabbitmq-azure-services.yaml

In [ ]:
kubectl apply -f rabbitmq/rabbitmq-azure-statefulset.yaml

In [ ]:
kubectl get pods -l app=rabbitmq

example)

```
NAME         READY     STATUS    RESTARTS   AGE
rabbitmq-0   1/1       Running   0          6m
rabbitmq-1   1/1       Running   0          5m
rabbitmq-2   1/1       Running   0          4m
```

In [ ]:
kubectl exec rabbitmq-0 -- rabbitmqctl cluster_status

example)
```
Cluster status of node rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local ...
[{nodes,[{disc,['rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local',
                'rabbit@rabbitmq-1.rabbitmq.default.svc.cluster.local',
                'rabbit@rabbitmq-2.rabbitmq.default.svc.cluster.local']}]},
 {running_nodes,['rabbit@rabbitmq-2.rabbitmq.default.svc.cluster.local',
                 'rabbit@rabbitmq-1.rabbitmq.default.svc.cluster.local',
                 'rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local']},
 {cluster_name,<<"rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local">>},
 {partitions,[]},
 {alarms,[{'rabbit@rabbitmq-2.rabbitmq.default.svc.cluster.local',[]},
          {'rabbit@rabbitmq-1.rabbitmq.default.svc.cluster.local',[]},
          {'rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local',[]}]}]
```

### change password of `guest`

In [ ]:
kubectl exec rabbitmq-0 -- rabbitmqctl change_password guest $(randomstr32)

### register users to RabbitMQ

In [ ]:
for e in $(env); do
  if [[ "${e}" =~ ^MQTT__([[:alnum:]_-]+)=([[:alnum:]_-]+)$ ]]; then
    username=${BASH_REMATCH[1]}
    password=${BASH_REMATCH[2]}
    
    kubectl exec rabbitmq-0 -- rabbitmqctl add_user ${username} ${password}
    kubectl exec rabbitmq-0 -- rabbitmqctl set_permissions -p / ${username} ".*" ".*" ".*"
  fi
done

In [ ]:
kubectl exec rabbitmq-0 -- rabbitmqctl list_users

example)
```
Listing users ...
guest	[administrator]
iotagent	[]
```

### register DNS A Record for RabbitMQ

**confirm that "EXTERNAL-IP" is assigned**

In [ ]:
kubectl get services -l app=rabbitmq -l service=mqtts

expected)
```
NAME             TYPE           CLUSTER-IP   EXTERNAL-IP   PORT(S)          AGE
rabbitmq-mqtts   LoadBalancer   10.0.96.7    XX.XX.XX.XX   8883:30299/TCP   21m
```

In [ ]:
MQTTS_IPADDR=$(kubectl get services -l app=rabbitmq -l service=mqtts -o jsonpath='{.items[0].status.loadBalancer.ingress[0].ip}')
az network dns record-set a add-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "mqtt" --ipv4-address "${MQTTS_IPADDR}"

In [ ]:
nslookup mqtt.${DOMAIN}

In [ ]:
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${PJ_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /test -m "test"

example)
```
Client mosqpub|52150-Nobuyukin sending CONNECT
Client mosqpub|52150-Nobuyukin received CONNACK
Client mosqpub|52150-Nobuyukin sending PUBLISH (d0, q0, r0, m1, '/test', ... (4 bytes))
Client mosqpub|52150-Nobuyukin sending DISCONNECT
```

## start mondodb cluster on AKS
[mongodb](https://www.mongodb.com/)

create 'mongodb' takes a few minutes.

In [ ]:
helm install --name mongodb -f mongodb/mongodb-replicaset-values-azure.yaml stable/mongodb-replicaset

In [ ]:
kubectl get PersistentVolumeClaims -l release=mongodb -l app=mongodb-replicaset

example)
```
NAME                              STATUS    VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS      AGE
mongodb-storage-claim-mongodb-0   Bound     pvc-386ac3ba-856e-11e8-a3b4-026b849f8c40   30Gi       RWO            managed-premium   5m
mongodb-storage-claim-mongodb-1   Bound     pvc-88455199-856e-11e8-a3b4-026b849f8c40   30Gi       RWO            managed-premium   3m
mongodb-storage-claim-mongodb-2   Bound     pvc-b74ec32c-856e-11e8-a3b4-026b849f8c40   30Gi       RWO            managed-premium   2m
```

In [ ]:
kubectl get statefulsets -l release=mongodb -l app=mongodb-replicaset

example)
```
NAME      READY   AGE
mongodb   3/3     5m32s
```

In [ ]:
kubectl get pods -l release=mongodb -l app=mongodb-replicaset

example)
```
NAME        READY   STATUS    RESTARTS   AGE
mongodb-0   1/1     Running   0          5m55s
mongodb-1   1/1     Running   0          4m
mongodb-2   1/1     Running   0          2m27s
```

In [ ]:
kubectl get services -l release=mongodb -l app=mongodb-replicaset

example)
```
NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)     AGE
mongodb          ClusterIP   None         <none>        27017/TCP   6m15s
mongodb-client   ClusterIP   None         <none>        27017/TCP   6m15s
```

In [ ]:
kubectl exec mongodb-0 -c mongodb-replicaset -- mongo --eval 'printjson(rs.status().members.map(function(e) {return {name: e.name, stateStr:e.stateStr};}))'

expected)
```
MongoDB shell version v4.1.13
connecting to: mongodb://127.0.0.1:27017/?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("2508d7c2-8857-4228-8f1d-0fd1e6859b08") }
MongoDB server version: 4.1.13
[
	{
		"name" : "mongodb-0.mongodb.default.svc.cluster.local:27017",
		"stateStr" : "PRIMARY"
	},
	{
		"name" : "mongodb-1.mongodb.default.svc.cluster.local:27017",
		"stateStr" : "SECONDARY"
	},
	{
		"name" : "mongodb-2.mongodb.default.svc.cluster.local:27017",
		"stateStr" : "SECONDARY"
	}
]
```

## start ambassador on AKS

[ambassador](https://www.getambassador.io/)

### register cert files of ambassador to kubernetes secrets

In [ ]:
kubectl create secret tls ambassador-certs --cert=${PJ_ROOT}/secrets/live/${DOMAIN}/fullchain.pem --key=${PJ_ROOT}/secrets/live/${DOMAIN}/privkey.pem

### start ambassador

In [ ]:
kubectl apply -f ambassador/ambassador-azure-services.yaml

In [ ]:
kubectl apply -f ambassador/ambassador-deployment.yaml

In [ ]:
kubectl get pods -l app=ambassador

example)
```
NAME                         READY     STATUS    RESTARTS   AGE
ambassador-cf99ff6bb-45jpl   2/2       Running   0          39s
ambassador-cf99ff6bb-8kht7   2/2       Running   0          39s
ambassador-cf99ff6bb-lv2tk   2/2       Running   0          39s
```

**confirm that "EXTERNAL-IP" is assigned**

In [ ]:
kubectl get services -l app=ambassador

example)
```
NAME         TYPE           CLUSTER-IP    EXTERNAL-IP   PORT(S)                      AGE
ambassador   LoadBalancer   10.0.191.59   YY.YY.YY.YY   443:30357/TCP,80:32755/TCP   4m
```

### register DNS A Record for ambassador

In [ ]:
HTTPS_IPADDR=$(kubectl get services -l app=ambassador -o json | jq '.items[0].status.loadBalancer.ingress[0].ip' -r)
az network dns record-set a add-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "api" --ipv4-address "${HTTPS_IPADDR}"

In [ ]:
nslookup api.${DOMAIN}

In [ ]:
curl -i https://api.${DOMAIN}

expected)
```
HTTP/1.1 404 Not Found
date: Fri, 25 May 2018 00:47:41 GMT
server: envoy
content-length: 0
```